# LEKCE 3

# Spojování, agregace a řazení tučňáků

## Načtěte dataset Palmer Penguins
Můžete jej stáhnout z GitHub [repozitáře](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv).

Použijte knihovnu pandas, metodu `read_csv()` a data uložte do dataframu nazváného data.

#### Dokumentace k datasetu “Penguins”

Tento dataset obsahuje údaje o třech druzích tučňáků, kteří se vyskytují v oblasti Palmerovy stanice na Antarktidě. Dataset je často využíván pro výukové účely a analýzy dat.

Dataset obsahuje měření různých fyzických charakteristik tučňáků ze tří různých ostrovů. Data zahrnují informace o délce zobáku, hloubce zobáku, délce ploutví, hmotnosti těla a dalších proměnných. Měření byla provedena na třech druzích tučňáků: Adélie, Gentoo, a Chinstrap.

**Sloupce datasetu:**

- species – Druh tučňáka (Adélie, Gentoo, Chinstrap)
- island – Ostrov, na kterém byl tučňák pozorován (Biscoe, Dream, Torgersen)
- bill_length_mm – Délka zobáku v milimetrech
- bill_depth_mm – Hloubka zobáku v milimetrech
- flipper_length_mm – Délka ploutve v milimetrech
- body_mass_g – Hmotnost těla tučňáka v gramech
- sex – Pohlaví tučňáka (Male nebo Female)
- year – Rok, kdy bylo měření provedeno

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv"
data = pd.read_csv(url)

In [14]:
# data.mean()
prumer = data.groupby("species")["body_mass_g"].mean()
prumer

species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

### Jaké jsou průměrné hodnoty hmotnost tučňáků podle jejich druhu?

In [3]:
data.groupby("species")["body_mass_g"].mean()

species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

### Který druh tučňáka má největší maximální hmotnost a jaká je tato hmotnost?

Nejprve seskupte data podle druhu tučňáka pomocí `groupby()`, poté použijte agregaci `max()` na sloupec s hmotností.  
Nakonec použijte metodu `sort_values()` pro seřazení výsledků sestupně a získejte první řádek pro odpověď.

In [4]:
data.groupby("species")["body_mass_g"].max().sort_values(ascending=False).head(1)

species
Gentoo    6300.0
Name: body_mass_g, dtype: float64

### Jaký je počet tučňáků podle jejich pohlaví v každém druhu? 

In [5]:
data.groupby(["species", "sex"]).size()
# data.groupby(["species", "sex"])["sex"].size()

species    sex   
Adelie     female    73
           male      73
Chinstrap  female    34
           male      34
Gentoo     female    58
           male      61
dtype: int64

### Jaké jsou minimální a maximální hodnoty délky křídel tučňáků a jak se liší mezi druhy? Který druh má nejmenší rozdíl mezi minimální a maximální délkou křídel?

Seskupte data podle druhu tučňáka a použijte metody `min()` a `max()` pro zjištění minimálních a maximálních hodnot délky křídel.  
Výsledky můžete zobrazit pomocí metody `agg()`, kde specifikujete obě agregační funkce.

In [6]:
# data.groupby("species")["flipper_length_mm"].max()
# data.groupby("species")["flipper_length_mm"].min()

data.groupby("species").agg({'flipper_length_mm': ['min', 'max']})

flipper_length_mm       
                        min    max
species                           
Adelie                172.0  210.0
Chinstrap             178.0  212.0
Gentoo                203.0  231.0

In [7]:
## MULTIINDEX

# Výhoda: Tento přístup je vhodný, pokud chceš provádět agregace nad více sloupci nebo pokud potřebuješ přístup ke sloupcům přes víceúrovňový index.
# Nevýhoda: Multiindex může být složitější na manipulaci a není vždy přehledný, pokud máš více agregovaných sloupců.
aggregated = data.groupby("species").agg({"flipper_length_mm": ["min", "max"]})

aggregated["diff"] = aggregated[(
    "flipper_length_mm", "max")] - aggregated[("flipper_length_mm", "min")]
aggregated.sort_values(by="diff")


flipper_length_mm         diff
                        min    max      
species                                 
Gentoo                203.0  231.0  28.0
Chinstrap             178.0  212.0  34.0
Adelie                172.0  210.0  38.0

In [8]:
# Výhoda: Tento přístup je přehlednější a lépe se s ním pracuje, zejména pokud potřebuješ přístup k agregovaným sloupcům s jednoduchými názvy.
# Výhoda: Názvy sloupců můžeš při agregaci rovnou definovat, což ti umožní přesně říct, jak se mají jednotlivé výsledky jmenovat.
# Nevýhoda: Méně flexibilní při agregaci více sloupců současně (na rozdíl od slovníkového přístupu v prvním přístupu).
aggregated = data.groupby("species").agg(
    min_flipper_length=("flipper_length_mm", "min"),
    max_flipper_length=("flipper_length_mm", "max"),
)

aggregated["diff"] = aggregated["max_flipper_length"] - aggregated["min_flipper_length"]
aggregated.sort_values(by="diff")

,min_flipper_length,max_flipper_length,diff
species,,,
Gentoo,203.0,231.0,28.0
Chinstrap,178.0,212.0,34.0
Adelie,172.0,210.0,38.0


### Jaký je medián hmotnosti tučňáků a liší se tento medián mezi jednotlivými ostrovy, kde tučňáci žijí? Na kterém ostrově je medián hmotnosti největší?

Data seskupte podle názvu ostrova a použijte agregační funkci `median()` pro výpočet mediánu hmotnosti tučňáků na každém ostrově.

In [9]:
data.groupby("island")["body_mass_g"].median()

island
Biscoe       4775.0
Dream        3687.5
Torgersen    3700.0
Name: body_mass_g, dtype: float64

### Existují v datasetu chybějící hodnoty? Pokud ano, ve kterých sloupcích?

In [10]:
data.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

### Které druhy tučňáků mají průměrnou délku křídel větší než 200 mm?

In [11]:
agg_series = data.groupby(["species"])["flipper_length_mm"].mean()
agg_series[agg_series >= 200]

species
Gentoo    217.186992
Name: flipper_length_mm, dtype: float64

In [12]:
agg_df = data.groupby(["species"])["flipper_length_mm"].mean().to_frame()
agg_df[agg_df["flipper_length_mm"] >= 200]

,flipper_length_mm
species,
Gentoo,217.186992
